# Simple GNN Implementation (Phase 1) 

This is the initial GNN implementation using encoders and decoders. 


In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install --verbose torch-scatter
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.0.0+cu118
Using pip 23.0.1 from /usr/local/lib/python3.9/dist-packages/pip (python 3.9)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

# Tutorial 6 - Example
Graph AutoEncoders GAE &  
Variational Graph Autoencoders VGAE    

[Tutorial 6 paper](https://arxiv.org/pdf/1611.07308.pdf)  
[Tutorial 6 code](https://github.com/rusty1s/pytorch_geometric/blob/master/examples/autoencoder.py)

## Graph AutoEncoder GAE

### Load the data

In [ ]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
dataset.data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])

In [ ]:
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = None
data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327])

In [ ]:
data = train_test_split_edges(data)

In [ ]:
data

Data(x=[3327, 3703], y=[3327], val_pos_edge_index=[2, 227], test_pos_edge_index=[2, 455], train_pos_edge_index=[2, 7740], train_neg_adj_mask=[3327, 3327], val_neg_edge_index=[2, 227], test_neg_edge_index=[2, 455])

### Define the Encoder

In [ ]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)
    

### Define the Autoencoder

In [ ]:
from torch_geometric.nn import GAE

In [ ]:
# parameters
out_channels = 2
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    #if args.variational:
    #   loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)



In [ ]:
for epoch in range(1, epochs + 1):
    loss = train()

    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 001, AUC: 0.5968, AP: 0.6404
Epoch: 002, AUC: 0.6227, AP: 0.6620
Epoch: 003, AUC: 0.6253, AP: 0.6692
Epoch: 004, AUC: 0.6258, AP: 0.6700
Epoch: 005, AUC: 0.6227, AP: 0.6641
Epoch: 006, AUC: 0.6231, AP: 0.6644
Epoch: 007, AUC: 0.6237, AP: 0.6653
Epoch: 008, AUC: 0.6249, AP: 0.6670
Epoch: 009, AUC: 0.6264, AP: 0.6687
Epoch: 010, AUC: 0.6276, AP: 0.6707
Epoch: 011, AUC: 0.6281, AP: 0.6731
Epoch: 012, AUC: 0.6282, AP: 0.6746
Epoch: 013, AUC: 0.6282, AP: 0.6761
Epoch: 014, AUC: 0.6279, AP: 0.6778
Epoch: 015, AUC: 0.6270, AP: 0.6792
Epoch: 016, AUC: 0.6266, AP: 0.6815
Epoch: 017, AUC: 0.6253, AP: 0.6830
Epoch: 018, AUC: 0.6245, AP: 0.6847
Epoch: 019, AUC: 0.6239, AP: 0.6865
Epoch: 020, AUC: 0.6232, AP: 0.6880
Epoch: 021, AUC: 0.6231, AP: 0.6899
Epoch: 022, AUC: 0.6228, AP: 0.6914
Epoch: 023, AUC: 0.6228, AP: 0.6924
Epoch: 024, AUC: 0.6229, AP: 0.6933
Epoch: 025, AUC: 0.6235, AP: 0.6944
Epoch: 026, AUC: 0.6240, AP: 0.6949
Epoch: 027, AUC: 0.6248, AP: 0.6961
Epoch: 028, AUC: 0.6257, AP:

In [ ]:
Z = model.encode(x, train_pos_edge_index)
Z

tensor([[-0.2001,  0.4399],
        [ 0.4931, -1.0101],
        [-0.3626,  0.9303],
        ...,
        [ 0.0893, -0.5191],
        [-0.3680,  0.8080],
        [-0.3584,  1.0265]], grad_fn=<AddBackward0>)

## Are the results (AUC) and (AP) easy to read and compare?

# Use Tensorboard

In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
# parameters
out_channels = 2
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### Import tensorboard

#### Installation: (if needed) "pip install tensorboard"

In [ ]:
writer = SummaryWriter('runs/GAE1_experiment_'+'2d_100_epochs')

In [ ]:
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.7885, AP: 0.7867
Epoch: 002, AUC: 0.7885, AP: 0.7866
Epoch: 003, AUC: 0.7876, AP: 0.7861
Epoch: 004, AUC: 0.7871, AP: 0.7859
Epoch: 005, AUC: 0.7874, AP: 0.7860
Epoch: 006, AUC: 0.7875, AP: 0.7859
Epoch: 007, AUC: 0.7878, AP: 0.7861
Epoch: 008, AUC: 0.7877, AP: 0.7859
Epoch: 009, AUC: 0.7870, AP: 0.7855
Epoch: 010, AUC: 0.7861, AP: 0.7850
Epoch: 011, AUC: 0.7856, AP: 0.7845
Epoch: 012, AUC: 0.7859, AP: 0.7847
Epoch: 013, AUC: 0.7861, AP: 0.7846
Epoch: 014, AUC: 0.7859, AP: 0.7842
Epoch: 015, AUC: 0.7856, AP: 0.7840
Epoch: 016, AUC: 0.7850, AP: 0.7838
Epoch: 017, AUC: 0.7842, AP: 0.7832
Epoch: 018, AUC: 0.7832, AP: 0.7828
Epoch: 019, AUC: 0.7824, AP: 0.7822
Epoch: 020, AUC: 0.7823, AP: 0.7820
Epoch: 021, AUC: 0.7829, AP: 0.7823
Epoch: 022, AUC: 0.7831, AP: 0.7819
Epoch: 023, AUC: 0.7825, AP: 0.7813
Epoch: 024, AUC: 0.7821, AP: 0.7811
Epoch: 025, AUC: 0.7819, AP: 0.7812
Epoch: 026, AUC: 0.7810, AP: 0.7808
Epoch: 027, AUC: 0.7807, AP: 0.7807
Epoch: 028, AUC: 0.7808, AP:

## Graph Variational AutoEncoder (GVAE)

In [ ]:
from torch_geometric.nn import VGAE
!pip install --verbose torch-scatter

Using pip 23.0.1 from /usr/local/lib/python3.9/dist-packages/pip (python 3.9)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)


class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [ ]:
out_channels = 2
num_features = dataset.num_features
epochs = 300


model = VGAE(VariationalGCNEncoder(num_features, out_channels))  # new line

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    
    loss = loss + (1 / data.num_nodes) * model.kl_loss()  # new line
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [ ]:
writer = SummaryWriter('runs/VGAE_experiment_'+'2d_100_epochs')

for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.5754, AP: 0.5951
Epoch: 002, AUC: 0.5847, AP: 0.6056
Epoch: 003, AUC: 0.6097, AP: 0.6272
Epoch: 004, AUC: 0.6135, AP: 0.6302
Epoch: 005, AUC: 0.6156, AP: 0.6323
Epoch: 006, AUC: 0.6164, AP: 0.6327
Epoch: 007, AUC: 0.6169, AP: 0.6331
Epoch: 008, AUC: 0.6173, AP: 0.6345
Epoch: 009, AUC: 0.6175, AP: 0.6349
Epoch: 010, AUC: 0.6170, AP: 0.6352
Epoch: 011, AUC: 0.6168, AP: 0.6355
Epoch: 012, AUC: 0.6170, AP: 0.6358
Epoch: 013, AUC: 0.6169, AP: 0.6356
Epoch: 014, AUC: 0.6165, AP: 0.6355
Epoch: 015, AUC: 0.6162, AP: 0.6350
Epoch: 016, AUC: 0.6156, AP: 0.6339
Epoch: 017, AUC: 0.6152, AP: 0.6335
Epoch: 018, AUC: 0.6152, AP: 0.6334
Epoch: 019, AUC: 0.6145, AP: 0.6329
Epoch: 020, AUC: 0.6141, AP: 0.6326
Epoch: 021, AUC: 0.6137, AP: 0.6322
Epoch: 022, AUC: 0.6132, AP: 0.6318
Epoch: 023, AUC: 0.6128, AP: 0.6316
Epoch: 024, AUC: 0.6119, AP: 0.6310
Epoch: 025, AUC: 0.6114, AP: 0.6304
Epoch: 026, AUC: 0.6105, AP: 0.6296
Epoch: 027, AUC: 0.6099, AP: 0.6289
Epoch: 028, AUC: 0.6089, AP: